In [1]:
import pandas as pd
import numpy as np

In [11]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

In [3]:
data = pd.read_csv("Data/пп_станции.xlsx - Лист1.csv")
data.head()

,Станция,Номер линии,Дата,4/3/2024,4/2/2024,4/1/2024,3/31/2024,3/30/2024,3/29/2024,3/28/2024,3/27/2024,3/26/2024,3/25/2024,3/24/2024,3/23/2024,3/22/2024,3/21/2024,3/20/2024,3/19/2024,3/18/2024,3/17/2024,3/16/2024,3/15/2024,3/14/2024,3/13/2024,3/12/2024,3/11/2024,3/10/2024,3/9/2024,3/8/2024,3/7/2024,3/6/2024,3/5/2024,3/4/2024,3/3/2024,3/2/2024,3/1/2024,2/29/2024,2/28/2024,2/27/2024,2/26/2024,2/25/2024,2/24/2024,2/23/2024,2/22/2024,2/21/2024,2/20/2024,2/19/2024,2/18/2024,2/17/2024,2/16/2024,2/15/2024,2/14/2024,2/13/2024,2/12/2024,2/11/2024,2/10/2024,2/9/2024,2/8/2024,2/7/2024,2/6/2024,2/5/2024,2/4/2024,2/3/2024,2/2/2024,2/1/2024,1/31/2024,1/30/2024,1/29/2024,1/28/2024,1/27/2024,1/26/2024,1/25/2024,1/24/2024,1/23/2024,1/22/2024,1/21/2024,1/20/2024,1/19/2024,1/18/2024,1/17/2024,1/16/2024,1/15/2024,1/14/2024,1/13/2024,1/12/2024,1/11/2024,1/10/2024,1/9/2024,1/8/2024,1/7/2024,1/6/2024,1/5/2024,1/4/2024,1/3/2024,1/2/2024,1/1/2024
0,Б.Рокоссовского,8,СОКОЛЬНИЧЕСКАЯ,16671,15113,2905,13998,12742,1217,17624,12097,14831,7792,9609,1229,2135,6895,1041,18651,4943,5619,17125,19679,8482,3261,8065,17639,7450,2673,1088,5490,11985,5938,18694,6368,5575,17447,2239,2812,11670,5146,19224,10012,6214,8652,18346,13882,1888,18150,11240,12338,13991,18355,1248,4186,18087,14410,16187,9018,9734,4153,10858,3697,18317,17464,7277,4205,12204,3369,19395,1223,14351,2600,14185,7410,18924,10265,3061,1510,10798,19898,7645,17944,16665,14355,10499,18189,7280,4669,3812,5816,15740,13714,2561,11209,2722,4835
1,Черкизовская,8,СОКОЛЬНИЧЕСКАЯ,6248,5441,7991,14864,10015,9721,6623,13030,12372,17814,15379,2339,14576,18593,2588,6929,7277,3129,17656,1294,4426,4148,14293,16764,15338,6452,12030,8008,17510,16592,4520,18700,8816,2181,15752,14542,5213,4556,12789,5340,17570,3025,3865,8765,7515,17010,15248,3176,12324,15487,16393,12995,9768,16338,12248,17086,18639,14294,14153,12435,12251,2947,11779,14577,18868,17624,16820,11171,10935,2485,13333,11854,1667,5797,12029,1828,6468,12877,7645,15416,17947,9090,3849,17700,13486,5061,3663,11353,6924,6496,15991,7029,3999,15638
2,Преображенск. пл,8,СОКОЛЬНИЧЕСКАЯ,14094,1132,6556,7265,4001,10961,6852,13961,11940,2349,19378,14975,12584,3391,12367,7694,5643,14169,6267,7095,2167,7788,17924,9286,13476,14517,9040,12822,7241,8939,9775,10587,8814,19336,11912,8303,6590,1276,17036,11525,2127,3033,6349,4682,10830,16736,4526,19907,1634,6083,9522,15279,9890,19921,6404,6843,16351,12981,3417,11412,16221,17054,10516,15896,17579,14727,15152,8801,1530,7813,7735,10134,14017,8169,10657,12490,18196,5698,18122,10740,12682,9135,19574,4361,8176,9745,13281,15090,13309,8365,2811,5004,17068,17117
3,Сокольники СЛ,8,СОКОЛЬНИЧЕСКАЯ,18651,10083,6488,6409,9446,12051,15041,6793,3010,19196,17958,13304,19168,16050,8283,1348,8076,9769,13495,13672,16816,18654,7779,11842,4898,7845,8019,11014,7822,13541,19996,18196,13928,10224,2236,12909,9933,6320,5646,19848,17931,10014,11184,6686,16012,3537,10608,12939,19224,11326,18973,1272,6400,19425,1662,17534,19227,11321,1465,9050,17897,2104,4598,9385,2009,7977,7327,16742,9381,3279,15963,10741,6518,2690,16002,15438,18937,11864,19236,7969,12214,8669,7326,7890,3637,14001,12049,15972,15545,18759,5573,15017,14176,5465
4,Красносельская,8,СОКОЛЬНИЧЕСКАЯ,7391,19339,15424,10725,5965,14682,18134,2055,9929,9980,10764,5599,9113,4628,11462,18312,18938,3711,10294,13716,8803,1803,13435,2167,6980,3814,12569,11659,14564,15689,1267,13190,13247,18587,17385,19361,10486,5850,15293,11900,14056,17564,2791,19596,5429,4949,9723,14583,15674,7914,4116,5767,10763,17830,10085,8934,8231,17688,12619,8573,9204,11679,5864,3054,7617,12718,4241,16727,8937,13782,18008,14760,2836,1210,8202,17022,11939,7065,8814,7338,7490,3529,17303,8012,14194,7615,9321,16152,4581,7746,8936,11373,19561,1727


In [4]:
data.rename(columns={'Дата': 'Линия'}, inplace=True)

In [5]:
data.shape

(322, 97)

In [12]:
cleaned_data = data.drop_duplicates(subset=['Станция', 'Номер линии', 'Линия'], keep='first')
cleaned_data

,Станция,Номер линии,Линия,4/3/2024,4/2/2024,4/1/2024,3/31/2024,3/30/2024,3/29/2024,3/28/2024,3/27/2024,3/26/2024,3/25/2024,3/24/2024,3/23/2024,3/22/2024,3/21/2024,3/20/2024,3/19/2024,3/18/2024,3/17/2024,3/16/2024,3/15/2024,3/14/2024,3/13/2024,3/12/2024,3/11/2024,3/10/2024,3/9/2024,3/8/2024,3/7/2024,3/6/2024,3/5/2024,3/4/2024,3/3/2024,3/2/2024,3/1/2024,2/29/2024,2/28/2024,2/27/2024,2/26/2024,2/25/2024,2/24/2024,2/23/2024,2/22/2024,2/21/2024,2/20/2024,2/19/2024,2/18/2024,2/17/2024,2/16/2024,2/15/2024,2/14/2024,2/13/2024,2/12/2024,2/11/2024,2/10/2024,2/9/2024,2/8/2024,2/7/2024,2/6/2024,2/5/2024,2/4/2024,2/3/2024,2/2/2024,2/1/2024,1/31/2024,1/30/2024,1/29/2024,1/28/2024,1/27/2024,1/26/2024,1/25/2024,1/24/2024,1/23/2024,1/22/2024,1/21/2024,1/20/2024,1/19/2024,1/18/2024,1/17/2024,1/16/2024,1/15/2024,1/14/2024,1/13/2024,1/12/2024,1/11/2024,1/10/2024,1/9/2024,1/8/2024,1/7/2024,1/6/2024,1/5/2024,1/4/2024,1/3/2024,1/2/2024,1/1/2024
0,Б.Рокоссовского,8,СОКОЛЬНИЧЕСКАЯ,16671,15113,2905,13998,12742,1217,17624,12097,14831,7792,9609,1229,2135,6895,1041,18651,4943,5619,17125,19679,8482,3261,8065,17639,7450,2673,1088,5490,11985,5938,18694,6368,5575,17447,2239,2812,11670,5146,19224,10012,6214,8652,18346,13882,1888,18150,11240,12338,13991,18355,1248,4186,18087,14410,16187,9018,9734,4153,10858,3697,18317,17464,7277,4205,12204,3369,19395,1223,14351,2600,14185,7410,18924,10265,3061,1510,10798,19898,7645,17944,16665,14355,10499,18189,7280,4669,3812,5816,15740,13714,2561,11209,2722,4835
1,Черкизовская,8,СОКОЛЬНИЧЕСКАЯ,6248,5441,7991,14864,10015,9721,6623,13030,12372,17814,15379,2339,14576,18593,2588,6929,7277,3129,17656,1294,4426,4148,14293,16764,15338,6452,12030,8008,17510,16592,4520,18700,8816,2181,15752,14542,5213,4556,12789,5340,17570,3025,3865,8765,7515,17010,15248,3176,12324,15487,16393,12995,9768,16338,12248,17086,18639,14294,14153,12435,12251,2947,11779,14577,18868,17624,16820,11171,10935,2485,13333,11854,1667,5797,12029,1828,6468,12877,7645,15416,17947,9090,3849,17700,13486,5061,3663,11353,6924,6496,15991,7029,3999,15638
2,Преображенск. пл,8,СОКОЛЬНИЧЕСКАЯ,14094,1132,6556,7265,4001,10961,6852,13961,11940,2349,19378,14975,12584,3391,12367,7694,5643,14169,6267,7095,2167,7788,17924,9286,13476,14517,9040,12822,7241,8939,9775,10587,8814,19336,11912,8303,6590,1276,17036,11525,2127,3033,6349,4682,10830,16736,4526,19907,1634,6083,9522,15279,9890,19921,6404,6843,16351,12981,3417,11412,16221,17054,10516,15896,17579,14727,15152,8801,1530,7813,7735,10134,14017,8169,10657,12490,18196,5698,18122,10740,12682,9135,19574,4361,8176,9745,13281,15090,13309,8365,2811,5004,17068,17117
3,Сокольники СЛ,8,СОКОЛЬНИЧЕСКАЯ,18651,10083,6488,6409,9446,12051,15041,6793,3010,19196,17958,13304,19168,16050,8283,1348,8076,9769,13495,13672,16816,18654,7779,11842,4898,7845,8019,11014,7822,13541,19996,18196,13928,10224,2236,12909,9933,6320,5646,19848,17931,10014,11184,6686,16012,3537,10608,12939,19224,11326,18973,1272,6400,19425,1662,17534,19227,11321,1465,9050,17897,2104,4598,9385,2009,7977,7327,16742,9381,3279,15963,10741,6518,2690,16002,15438,18937,11864,19236,7969,12214,8669,7326,7890,3637,14001,12049,15972,15545,18759,5573,15017,14176,5465
4,Красносельская,8,СОКОЛЬНИЧЕСКАЯ,7391,19339,15424,10725,5965,14682,18134,2055,9929,9980,10764,5599,9113,4628,11462,18312,18938,3711,10294,13716,8803,1803,13435,2167,6980,3814,12569,11659,14564,15689,1267,13190,13247,18587,17385,19361,10486,5850,15293,11900,14056,17564,2791,19596,5429,4949,9723,14583,15674,7914,4116,5767,10763,17830,10085,8934,8231,17688,12619,8573,9204,11679,5864,3054,7617,12718,4241,16727,8937,13782,18008,14760,2836,1210,8202,17022,11939,7065,8814,7338,7490,3529,17303,8012,14194,7615,9321,16152,4581,7746,8936,11373,19561,1727
5,Комсомольск. СЛ,8,СОКОЛЬНИЧЕСКАЯ,8883,15442,15055,2531,7285,10377,11464,6108,13639,15436,17389,12214,19581,8934,9972,19826,3473,14382,7749,7140,10722,10091,15357,2730,5689,4351,16308,18628,8470,13682,13928,18224,10595,9983,17323,9170,9296,17462,16814,10759,9693,7878,12370,14083,7500,16260,11070,1

In [15]:
for index, row in cleaned_data.iterrows():
    print(row["Станция"], row["Линия"], " - ", row["Станция"], row["Линия"]) 

Б.Рокоссовского СОКОЛЬНИЧЕСКАЯ  -  Б.Рокоссовского СОКОЛЬНИЧЕСКАЯ
Черкизовская СОКОЛЬНИЧЕСКАЯ  -  Черкизовская СОКОЛЬНИЧЕСКАЯ
Преображенск. пл СОКОЛЬНИЧЕСКАЯ  -  Преображенск. пл СОКОЛЬНИЧЕСКАЯ
Сокольники СЛ СОКОЛЬНИЧЕСКАЯ  -  Сокольники СЛ СОКОЛЬНИЧЕСКАЯ
Красносельская СОКОЛЬНИЧЕСКАЯ  -  Красносельская СОКОЛЬНИЧЕСКАЯ
Комсомольск. СЛ СОКОЛЬНИЧЕСКАЯ  -  Комсомольск. СЛ СОКОЛЬНИЧЕСКАЯ
Красные ворота СОКОЛЬНИЧЕСКАЯ  -  Красные ворота СОКОЛЬНИЧЕСКАЯ
Чистые пруды СОКОЛЬНИЧЕСКАЯ  -  Чистые пруды СОКОЛЬНИЧЕСКАЯ
Лубянка СОКОЛЬНИЧЕСКАЯ  -  Лубянка СОКОЛЬНИЧЕСКАЯ
Охотный ряд СОКОЛЬНИЧЕСКАЯ  -  Охотный ряд СОКОЛЬНИЧЕСКАЯ
Кропоткинская СОКОЛЬНИЧЕСКАЯ  -  Кропоткинская СОКОЛЬНИЧЕСКАЯ
Парк культуры СЛ СОКОЛЬНИЧЕСКАЯ  -  Парк культуры СЛ СОКОЛЬНИЧЕСКАЯ
Фрунзенская СОКОЛЬНИЧЕСКАЯ  -  Фрунзенская СОКОЛЬНИЧЕСКАЯ
Спортивная СОКОЛЬНИЧЕСКАЯ  -  Спортивная СОКОЛЬНИЧЕСКАЯ
Воробьёвы горы СОКОЛЬНИЧЕСКАЯ  -  Воробьёвы горы СОКОЛЬНИЧЕСКАЯ
Университет СОКОЛЬНИЧЕСКАЯ  -  Университет СОКОЛЬНИЧЕСКАЯ
Пр-т Вернадск.СЛ